In [0]:
import keras
keras.__version__

In [0]:
!wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
!unzip jena_climate_2009_2016.csv

In [0]:
import os

data_dir = ''
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')

f = open(fname)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

print(header)
print(len(lines))

In [0]:
import numpy as np

float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
#    print(i,line)
    values = [float(x) for x in line.split(',')[1:]]
#    print(i,values)
    float_data[i, :] = values

In [0]:
from matplotlib import pyplot as plt

temp = float_data[:, 1]  # temperature (in degrees Celsius)
plt.plot(range(len(temp)), temp)
plt.show()

In [0]:
mean = float_data[:400000].mean(axis=0)
float_data -= mean
std = float_data[:400000].std(axis=0)
float_data /= std

In [0]:
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=1):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

In [0]:
lookback = 72 #12h
step = 6
delay = 36 #6h
batch_size = 128

train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=400000,
                      shuffle=True,
                      step=step, 
                      batch_size=batch_size)
val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=400001,
                    max_index=410000,
                    step=step,
                    batch_size=batch_size)
test_gen = generator(float_data,
                     lookback=lookback,
                     delay=delay,
                     min_index=410001,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

# This is how many steps to draw from `val_gen`
# in order to see the whole validation set:
val_steps = (410000 - 400001 - lookback) // batch_size

# This is how many steps to draw from `test_gen`
# in order to see the whole test set:
test_steps = (len(float_data) - 410001 - lookback) // batch_size

In [0]:
###################  No.1

In [0]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model1 = Sequential()
model1.add(layers.GRU(16, input_shape=(None, float_data.shape[-1])))
model1.add(layers.Dense(1))

model1.compile(optimizer=RMSprop(), loss='mae')

In [0]:
model1.summary()

In [0]:
#mkdir my_log_dir

In [0]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience = 1
    )
   ,
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,
        patience=2
    )
] 

In [0]:

history1 = model1.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=20,
                              callbacks = callbacks_list,
                              validation_data=val_gen,
                              validation_steps=val_steps)

In [0]:
loss = history1.history['loss']
val_loss = history1.history['val_loss']

epochs = range(len(loss))

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [0]:

predict1 = model1.predict_generator(test_gen,steps=test_steps)

In [0]:
predicted_temp1 = predict1  #* std[1]  + mean[1]

In [0]:
print(test_steps)

In [0]:
len(predicted_temp1)

In [0]:
true_temp = []
for step in range(test_steps):
    samples, targets = next(test_gen)    
    true_temp.extend(targets)

#print(true_temp)

In [0]:
len(true_temp)

In [0]:
# true_temp =  float_data[len(lines)-len(predicted_temp1)+1:, 1] 

In [0]:


plt.figure()
plt.plot(true_temp, 'g', label='True_temp')

plt.plot(predicted_temp1, 'b', label='Test_predict')
plt.title('Test_predict')
plt.legend()

plt.show()

In [0]:
MAE1= np.mean(np.abs(predicted_temp1 - true_temp))
MAE1

In [0]:
#################### No2

In [0]:
#from keras.models import Sequential
#from keras import layers
#from keras.optimizers import RMSprop

model2 = Sequential()
model2.add(layers.GRU(16,
                     dropout=0.1,
                     recurrent_dropout=0.2,
                     input_shape=(None, float_data.shape[-1])))
model2.add(layers.Dense(1))

model2.compile(optimizer=RMSprop(), loss='mae')
history2 = model2.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=40,
                              callbacks = callbacks_list,
                              validation_data=val_gen,
                              validation_steps=val_steps)

In [0]:
model2.summary()

In [0]:
loss = history2.history['loss']
val_loss = history2.history['val_loss']

epochs = range(len(loss))

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [0]:
predict2 = model2.predict_generator(test_gen, steps=test_steps)
predicted_temp2 = predict2 

In [0]:
plt.figure()
plt.plot(true_temp, 'g', label='True_temp')

plt.plot(predicted_temp2, 'b', label='Test_predict 2')
plt.title('Test_predict 2')
plt.legend()

plt.show()

In [0]:
MAE2= np.mean(np.abs(predicted_temp2 - true_temp))
MAE2